Libraries setup

In [20]:
#!pip install ortools osrm folium

In [21]:
#import osrm
import requests
import json
import numpy as np

print("✅ OSRM libraries installed!")

✅ OSRM libraries installed!


In [22]:

all_locations = [
    ("Warehouse", 31.522334, 72.576500),
    ('Shop 1', 31.570639, 72.889028),
    ('Shop 2', 31.591778, 72.836139),
    ('Shop 3', 31.495, 72.85875),
    ('Shop 4', 31.653, 72.865278),
    ('Shop 5', 31.677139, 72.909444),
    ('Shop 6', 31.480528, 72.652944),
    ('Shop 7', 31.431111, 72.745694),
    ('Shop 8', 31.644861, 72.818),
    ('Shop 9', 31.643861, 72.636389),
    ('Shop 10', 31.613639, 72.455833),
    ('Shop 11', 31.613806, 72.750056),
    ('Shop 12', 31.431444, 72.496167),
    ('Shop 13', 31.392056, 72.77875),
    ('Shop 14', 31.454667, 72.798083),
    ('Shop 15', 31.636528, 72.503222),
    ('Shop 16', 31.597333, 72.611944),
    ('Shop 17', 31.536472, 72.535139),
    ('Shop 18', 31.482861, 72.705389),
    ('Shop 19', 31.57825, 72.667722),
    ('Shop 20', 31.659056, 72.539472),
]

all_demands = [0, 40, 20, 30, 10, 12, 15, 18, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Filter out locations with zero demand (except warehouse at index 0)
locations = []
demands = []

for i, (location, demand) in enumerate(zip(all_locations, all_demands)):
    if i == 0 or demand > 0:  # Keep warehouse (index 0) and locations with demand > 0
        locations.append(location)
        demands.append(demand)

print("📍 Setup complete:")
print(f"  Total locations: {len(all_locations)}")
print(f"  Active locations (with demand): {len(locations)}")
print(f"  Filtered out: {len(all_locations) - len(locations)} locations")
print(f"  Total demand: {sum(demands)} bags")

print("\n📦 Active locations:")
for i, (name, _, _) in enumerate(locations):
    print(f"  {name}: {demands[i]} bags")

📍 Setup complete:
  Total locations: 21
  Active locations (with demand): 9
  Filtered out: 12 locations
  Total demand: 165 bags

📦 Active locations:
  Warehouse: 0 bags
  Shop 1: 40 bags
  Shop 2: 20 bags
  Shop 3: 30 bags
  Shop 4: 10 bags
  Shop 5: 12 bags
  Shop 6: 15 bags
  Shop 7: 18 bags
  Shop 8: 20 bags


In [73]:


# Vehicle fleet: (count, capacity in bags, cost_per_km)
# Example heterogenous fleet: 2 trucks and 2 vans
# Vehicle fleet: (count, capacity in bags, cost_per_km, rent_limit_km, fixed_rent)
# rent_limit_km: 0 for owned vehicles, distance limit for rented vehicles
# fixed_rent: 0 for owned vehicles, fixed rental cost for rented vehicles
vehicles = [
    {"count": 1, "capacity": 200, "cost_per_km": 100.0, "rent_limit_km": 0, "fixed_rent": 0},      # Owned big truck
    {"count": 4, "capacity": 50,  "cost_per_km": 50,   "rent_limit_km": 0, "fixed_rent": 0},      # Owned medium van
    {"count": 1, "capacity": 80,  "cost_per_km": 60,   "rent_limit_km": 100, "fixed_rent": 5000}, # Rented medium van: ₨5000 for 100km, then ₨60/km
    {"count": 1, "capacity": 50,  "cost_per_km": 50,   "rent_limit_km": 80, "fixed_rent": 2000},  # Rented small van: ₨3500 for 80km, then ₨50/km
]

print("🚛 Vehicle Fleet Configuration:")
for i, v in enumerate(vehicles):
    vehicle_type = "Owned" if v["rent_limit_km"] == 0 else "Rented"
    print(f"  Vehicle {i+1}: {vehicle_type} - {v['capacity']} bags capacity")
    if v["rent_limit_km"] == 0:
        print(f"    Cost: ₨{v['cost_per_km']}/km")
    else:
        print(f"    Rental: ₨{v['fixed_rent']} for {v['rent_limit_km']}km, then ₨{v['cost_per_km']}/km")
    print(f"    Count: {v['count']}")
    print()

🚛 Vehicle Fleet Configuration:
  Vehicle 1: Owned - 200 bags capacity
    Cost: ₨100.0/km
    Count: 1

  Vehicle 2: Owned - 50 bags capacity
    Cost: ₨50/km
    Count: 4

  Vehicle 3: Rented - 80 bags capacity
    Rental: ₨5000 for 100km, then ₨60/km
    Count: 1

  Vehicle 4: Rented - 50 bags capacity
    Rental: ₨2000 for 80km, then ₨50/km
    Count: 1



In [74]:
# Step 2: Demands - how much each location needs
#demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]  # warehouse needs 0, shops need fertilizer bags

print("\n📦 Demands (fertilizer bags):")
for i, (name, _, _) in enumerate(locations):
    print(f"  {name}: {demands[i]} bags")

total_demand = sum(demands)
print(f"\n📊 Total demand: {total_demand} bags")


📦 Demands (fertilizer bags):
  Warehouse: 0 bags
  Shop 1: 40 bags
  Shop 2: 20 bags
  Shop 3: 30 bags
  Shop 4: 10 bags
  Shop 5: 12 bags
  Shop 6: 15 bags
  Shop 7: 18 bags
  Shop 8: 20 bags

📊 Total demand: 165 bags


In [75]:
vehicle_specs = []
for v in vehicles:
    for _ in range(v["count"]):
        vehicle_specs.append((v["capacity"], v["cost_per_km"], v["rent_limit_km"], v["fixed_rent"]))
num_vehicles = len(vehicle_specs)

In [76]:
from math import radians, sin, cos, sqrt, atan2
def haversine_km(lat1, lon1, lat2, lon2):
    """Calculate real-world distance between two GPS points"""
    R = 6371.0  # Earth's radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [77]:
n = len(locations)
dist_km = [[0]*n for _ in range(n)]
road_factor = 1.25  # roads are ~25% longer than straight-line distance

print("\n📏 Distance Matrix (km):")
print("     ", end="")
for i in range(n):
    print(f"{i:5}", end="")
print()

for i in range(n):
    print(f"{i:2}: ", end="")
    for j in range(n):
        if i == j:
            dist_km[i][j] = 0
            print(f"{0:5.1f}", end="")
        else:
            dist_km[i][j] = haversine_km(locations[i][1], locations[i][2],
                                         locations[j][1], locations[j][2]) * road_factor


📏 Distance Matrix (km):
         0    1    2    3    4    5    6    7    8
 0:   0.0 1:   0.0 2:   0.0 3:   0.0 4:   0.0 5:   0.0 6:   0.0 7:   0.0 8:   0.0

In [78]:
import requests
import numpy as np

In [79]:
# Setup OSRM client - using public demo server
# For production, you'd run your own OSRM server
OSRM_URL = "http://router.project-osrm.org"

def get_osrm_matrix(locations):
    """Get distance and duration matrix using OSRM"""
    print(f"🌐 Getting OSRM matrix for {len(locations)} locations...")

    # Prepare coordinates for OSRM (lon,lat format)
    coordinates = []
    for name, lat, lon in locations:
        coordinates.append([lon, lat])  # OSRM uses lon,lat

    # Build OSRM table request URL
    coord_string = ";".join([f"{lon},{lat}" for lon, lat in coordinates])
    url = f"{OSRM_URL}/table/v1/driving/{coord_string}"

    params = {
        'sources': ';'.join([str(i) for i in range(len(locations))]),
        'destinations': ';'.join([str(i) for i in range(len(locations))]),
        'annotations': 'distance,duration'
    }

    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok':
            # Extract distance matrix (meters) and duration matrix (seconds)
            distances = np.array(data['distances']) / 1000  # Convert to km
            durations = np.array(data['durations']) / 60     # Convert to minutes

            print(f"✅ OSRM matrix retrieved successfully!")
            return distances, durations
        else:
            print(f"❌ OSRM error: {data['message']}")
            return None, None

    except Exception as e:
        print(f"❌ OSRM request failed: {e}")
        return None, None

def get_osrm_route(start_coords, end_coords):
    """Get detailed route between two points"""
    try:
        coord_string = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
        url = f"{OSRM_URL}/route/v1/driving/{coord_string}"

        params = {
            'overview': 'full',
            'geometries': 'geojson'
        }

        response = requests.get(url, params=params, timeout=15)
        data = response.json()

        if data['code'] == 'Ok':
            route = data['routes'][0]
            geometry = route['geometry']['coordinates']
            # Convert to lat,lon for folium
            path = [[coord[1], coord[0]] for coord in geometry]
            return path, route['distance']/1000, route['duration']/60
        else:
            return None, None, None

    except Exception as e:
        print(f"Route error: {e}")
        return None, None, None

# Test OSRM with our locations
print("🧪 Testing OSRM connectivity...")
osrm_distances, osrm_durations = get_osrm_matrix(locations)

🧪 Testing OSRM connectivity...
🌐 Getting OSRM matrix for 9 locations...
✅ OSRM matrix retrieved successfully!


In [80]:
import folium

In [81]:
if osrm_distances is not None:
    print("\n📊 OSRM Distance Matrix (km):")
    print("     ", end="")
    for i in range(len(locations)):
        print(f"{i:6}", end="")
    print()

    for i in range(len(locations)):
        print(f"{i:2}: ", end="")
        for j in range(len(locations)):
            print(f"{osrm_distances[i][j]:6.1f}", end="")
        print(f"  ({locations[i][0]})")

    # Convert to OR-Tools integer matrix
    osrm_dist_matrix = [[int(round(d * 100)) for d in row] for row in osrm_distances]
    print("✅ Using OSRM real driving distances!")

    # Compare with haversine
    print("\n📈 OSRM vs Haversine Comparison:")
    total_osrm = np.sum(osrm_distances)
    total_haversine = sum(sum(row) for row in dist_km)
    difference = ((total_osrm - total_haversine) / total_haversine) * 100
    print(f"   OSRM total: {total_osrm:.1f} km")
    print(f"   Haversine total: {total_haversine:.1f} km")
    print(f"   Difference: {difference:+.1f}% (OSRM more accurate)")

else:
    print("⚠️ OSRM failed, using haversine distances")
    osrm_dist_matrix = dist_matrix
    osrm_distances = dist_km


📊 OSRM Distance Matrix (km):
          0     1     2     3     4     5     6     7     8
 0:    0.0  37.4  30.8  32.1  32.8  36.4  15.0  24.1  26.9  (Warehouse)
 1:   37.4   0.0   6.6   9.2  11.5  17.3  37.5  24.7  14.8  (Shop 1)
 2:   30.8   6.6   0.0  11.6   8.7  14.5  37.0  27.3   8.1  (Shop 2)
 3:   32.1   9.2  11.6   0.0  20.3  26.1  28.5  15.7  19.8  (Shop 3)
 4:   32.8  11.5   8.7  20.3   0.0   5.8  39.0  43.1   5.9  (Shop 4)
 5:   36.4  17.3  14.5  26.1   5.8   0.0  42.6  46.7   9.5  (Shop 5)
 6:   15.0  37.5  37.0  28.5  39.0  42.6   0.0  13.9  33.1  (Shop 6)
 7:   24.1  24.7  27.3  15.7  43.1  46.7  13.9   0.0  37.2  (Shop 7)
 8:   26.9  14.8   8.1  19.8   5.9   9.5  33.1  37.2   0.0  (Shop 8)
✅ Using OSRM real driving distances!

📈 OSRM vs Haversine Comparison:
   OSRM total: 1702.6 km
   Haversine total: 1645.0 km
   Difference: +3.5% (OSRM more accurate)


In [ ]:
# Setup VRP with OSRM distances
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
manager = pywrapcp.RoutingIndexManager(len(osrm_distances), len(vehicle_specs), 0)
routing = pywrapcp.RoutingModel(manager)


# Add fixed costs after solving
def calculate_true_cost(route_distance, cost_per_km, rent_limit_km, fixed_rent):
    if rent_limit_km > 0:  # Rented vehicle
        if route_distance <= rent_limit_km:
            return fixed_rent  # Only pay fixed rental
        else:
            extra_distance = route_distance - rent_limit_km
            return fixed_rent + (extra_distance * cost_per_km)
    else:  # Owned vehicle
        return route_distance * cost_per_km

def make_consistent_cost_eval(cost_per_km, rent_limit_km, fixed_rent):
    def cost_eval(from_index, to_index):
        f = manager.IndexToNode(from_index)
        t = manager.IndexToNode(to_index)
        distance_km = osrm_distances[f][t]

        # Use variable cost rate for optimization
        cost = distance_km * cost_per_km
        return int(round(cost * 100))

    return cost_eval
# Fix the cost evaluator registration
for vid, (capacity, cost_per_km, rent_limit_km, fixed_rent) in enumerate(vehicle_specs):
    cost_evaluator = routing.RegisterTransitCallback(
        make_consistent_cost_eval(cost_per_km, rent_limit_km, fixed_rent)
    )
    routing.SetArcCostEvaluatorOfVehicle(cost_evaluator, vid)

# Add demand constraints
def demand_callback(from_index):
    node = manager.IndexToNode(from_index)
    return demands[node]

demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_cb_idx, 0, [spec[0] for spec in vehicle_specs], True, "Capacity")

# FIXED: Distance tracking with consistent units
def distance_callback(from_index, to_index):
    f = manager.IndexToNode(from_index)
    t = manager.IndexToNode(to_index)
    # Use KM and multiply by 100 for integer precision
    return int(round(osrm_distances[f][t] * 100))

distance_cb_idx = routing.RegisterTransitCallback(distance_callback)
routing.AddDimension(distance_cb_idx, 0, 50000, True, "Distance")  # 500km max

# FIXED: Distance limits with consistent units
distance_dimension = routing.GetDimensionOrDie("Distance")
for v in range(len(vehicle_specs)):
    capacity, cost_per_km, rent_limit_km, fixed_rent = vehicle_specs[v]
    if rent_limit_km > 0:  # Rented vehicle
        # Both in same units now (km * 100)
        max_distance = int(rent_limit_km * 100)
        distance_dimension.CumulVar(routing.End(v)).SetMax(max_distance)

# Solve
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.time_limit.seconds = 30  # Increased time limit

print("\n🔍 Solving VRP with FIXED units and logic...")
solution = routing.SolveWithParameters(search_parameters)


🔍 Solving VRP with FIXED units and logic...


### OSRM - Generate Routes

In [ ]:
# Add this function at the top of your results processing cell
def calculate_true_cost(route_distance, cost_per_km, rent_limit_km, fixed_rent):
    if rent_limit_km > 0:  # Rented vehicle
        if route_distance <= rent_limit_km:
            return fixed_rent  # Only pay fixed rental
        else:
            extra_distance = route_distance - rent_limit_km
            return fixed_rent + (extra_distance * cost_per_km)
    else:  # Owned vehicle
        return route_distance * cost_per_km

if solution:
    print("✅ OSRM-enhanced solution found!")
    osrm_routes = []

    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        load = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(next_index):
                route_distance += osrm_distances[node][manager.IndexToNode(next_index)]
            index = next_index

        if len(route) > 1:  # Vehicle is used
            capacity, cost_per_km, rent_limit_km, fixed_rent = vehicle_specs[v]

            # Calculate TRUE cost using the helper function
            true_cost = calculate_true_cost(route_distance, cost_per_km, rent_limit_km, fixed_rent)

            # ...existing path calculation code...

            vehicle_type = "Owned" if rent_limit_km == 0 else "Rented"
            
            osrm_routes.append({
                'vehicle_id': v + 1,
                'route': route,
                'full_path': full_path,
                'load': load,
                'capacity': capacity,
                'distance': route_distance,
                'cost': true_cost,  # Use TRUE cost here
                'cost_per_km': cost_per_km,
                'vehicle_type': vehicle_type,
                'rent_limit_km': rent_limit_km,
                'fixed_rent': fixed_rent
            })

            # Display route with TRUE cost breakdown
            route_names = [locations[idx][0] for idx in route]
            print(f"🚚 Vehicle {v+1} ({vehicle_type} - {capacity} bags, ₨{cost_per_km}/km):")
            print(f"   Route: {' → '.join(route_names)} → Warehouse")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km (OSRM real roads)")
            
            if rent_limit_km > 0:  # Rented vehicle
                print(f"   Rental: ₨{fixed_rent} for {rent_limit_km}km")
                if route_distance > rent_limit_km:
                    extra_distance = route_distance - rent_limit_km
                    extra_cost = extra_distance * cost_per_km
                    print(f"   Extra: {extra_distance:.1f}km × ₨{cost_per_km}/km = ₨{extra_cost:.2f}")
                    print(f"   Total Cost: ₨{fixed_rent} + ₨{extra_cost:.2f} = ₨{true_cost:.2f}")
                else:
                    print(f"   Total Cost: ₨{true_cost:.2f} (within rental limit)")
            else:  # Owned vehicle
                print(f"   Cost: {route_distance:.1f}km × ₨{cost_per_km}/km = ₨{true_cost:.2f}")
            
            print(f"   Cost Per bag: ₨{true_cost/load:.1f}")

    # Summary will now use TRUE costs
    print(f"\n📊 OSRM-ENHANCED SUMMARY:")
    total_cost = sum(r['cost'] for r in osrm_routes)  # Now uses true costs
    # ...rest of summary...
    total_distance = sum(r['distance'] for r in osrm_routes)
    owned_vehicles = len([r for r in osrm_routes if r['vehicle_type'] == 'Owned'])
    rented_vehicles = len([r for r in osrm_routes if r['vehicle_type'] == 'Rented'])

    print(f"   Vehicles used: {len(osrm_routes)}/{len(vehicle_specs)} ({owned_vehicles} owned, {rented_vehicles} rented)")
    print(f"   Total distance: {total_distance:.1f} km (real driving)")
    print(f"   Total cost: ₨{total_cost:.2f}")
    print(f"   Cost per bag: ₨{total_cost/total_demand:.2f}")

else:
    print("❌ No OSRM solution found!")
    osrm_routes = []

✅ OSRM-enhanced solution found!

🛣️ Getting real paths for Vehicle 4...
🚚 Vehicle 4 (Owned - 50 bags, ₨50/km):
   Route: Warehouse → Shop 7 → Shop 3 → Warehouse
   Load: 48/50 bags (96.0% full)
   Distance: 39.7 km (OSRM real roads)
   Cost: 39.7km × ₨50/km = ₨1987.33
   Cost Per bag: ₨41.4

🛣️ Getting real paths for Vehicle 6...
🚚 Vehicle 6 (Rented - 80 bags, ₨60/km):
   Route: Warehouse → Shop 2 → Shop 1 → Shop 6 → Warehouse
   Load: 75/80 bags (93.8% full)
   Distance: 74.9 km (OSRM real roads)
   Rental: ₨5000 for 100km
   Total Cost: ₨5000.00 (within rental limit)
   Cost Per bag: ₨66.7

🛣️ Getting real paths for Vehicle 7...
🚚 Vehicle 7 (Rented - 50 bags, ₨50/km):
   Route: Warehouse → Shop 8 → Shop 5 → Shop 4 → Warehouse
   Load: 42/50 bags (84.0% full)
   Distance: 42.2 km (OSRM real roads)
   Rental: ₨2000 for 80km
   Total Cost: ₨2000.00 (within rental limit)
   Cost Per bag: ₨47.6

📊 OSRM-ENHANCED SUMMARY:
   Vehicles used: 3/7 (1 owned, 2 rented)
   Total distance: 156.9 km

✅ OSRM-enhanced solution found!

🛣️ Getting real paths for Vehicle 2...
🚚 Vehicle 2 (Owned - 50 bags, ₨50/km):
   Route: Warehouse → Shop 1 → Warehouse
   Load: 40/50 bags (80.0% full)
   Distance: 37.4 km (OSRM real roads)
   Cost: 37.4km × ₨50/km = ₨1871.23
   Cost Per bag: ₨46.8

🛣️ Getting real paths for Vehicle 3...
🚚 Vehicle 3 (Owned - 50 bags, ₨50/km):
   Route: Warehouse → Shop 3 → Shop 2 → Warehouse
   Load: 50/50 bags (100.0% full)
   Distance: 43.7 km (OSRM real roads)
   Cost: 43.7km × ₨50/km = ₨2185.95
   Cost Per bag: ₨43.7

🛣️ Getting real paths for Vehicle 4...
🚚 Vehicle 4 (Owned - 50 bags, ₨50/km):
   Route: Warehouse → Shop 5 → Shop 4 → Shop 8 → Warehouse
   Load: 42/50 bags (84.0% full)
   Distance: 48.1 km (OSRM real roads)
   Cost: 48.1km × ₨50/km = ₨2404.82
   Cost Per bag: ₨57.3

🛣️ Getting real paths for Vehicle 5...
🚚 Vehicle 5 (Owned - 50 bags, ₨50/km):
   Route: Warehouse → Shop 6 → Shop 7 → Warehouse
   Load: 33/50 bags (66.0% full)
   Distance: 28.9 km (OSRM real roads)
   Cost: 28.9km × ₨50/km = ₨1446.82
   Cost Per bag: ₨43.8

📊 OSRM-ENHANCED SUMMARY:
   Vehicles used: 4/5 (4 owned, 0 rented)
   Total distance: 158.2 km (real driving)
   Total cost: ₨7908.82
   Cost per bag: ₨47.93

In [56]:
vehicles

[{'count': 1,
  'capacity': 200,
  'cost_per_km': 100.0,
  'rent_limit_km': 0,
  'fixed_rent': 0},
 {'count': 4,
  'capacity': 50,
  'cost_per_km': 50,
  'rent_limit_km': 0,
  'fixed_rent': 0},
 {'count': 1,
  'capacity': 80,
  'cost_per_km': 60,
  'rent_limit_km': 100,
  'fixed_rent': 5000},
 {'count': 1,
  'capacity': 50,
  'cost_per_km': 50,
  'rent_limit_km': 80,
  'fixed_rent': 2000}]

In [37]:
3389/165

20.53939393939394

### Generate Map

In [57]:
def create_osrm_map(locations, routes, demands, filename="osrm_map.html"):
    """Create map with real OSRM driving routes and save as HTML"""
    center_lat = sum(loc[1] for loc in locations) / len(locations)
    center_lon = sum(loc[2] for loc in locations) / len(locations)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add warehouse
    folium.Marker(
        [locations[0][1], locations[0][2]],
        popup=f"🏭 {locations[0][0]}",
        icon=folium.Icon(color='red', icon='home', prefix='fa')
    ).add_to(m)

    # Add shops
    for i, (name, lat, lon) in enumerate(locations[1:], 1):
        folium.Marker(
            [lat, lon],
            popup=f"🏪 {name}<br>Demand: {demands[i]} bags",
            icon=folium.Icon(color='blue', icon='shopping-cart', prefix='fa')
        ).add_to(m)

    # Add real driving routes
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred']

    for i, route in enumerate(routes):
        color = colors[i % len(colors)]

        folium.PolyLine(
            route['full_path'],
            color=color,
            weight=5,
            opacity=0.8,
            popup=f"🚚 Vehicle {route['vehicle_id']}<br>"
                  f"Real driving route<br>"
                  f"Distance: {route['distance']:.1f} km<br>"
                  f"Cost: ₨{route['cost']:.2f}",
        ).add_to(m)

    # Enhanced legend
    legend_html = f'''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 350px; height: 250px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:14px; padding: 10px">
    <h4>🗺️ OSRM-Enhanced Route Map</h4>
    <p>🏭 <span style="color:red">Red</span>: Warehouse</p>
    <p>🏪 <span style="color:blue">Blue</span>: Customer Shops</p>
    <p>🛣️ <span style="color:purple">Colored Lines</span>: Real Driving Routes</p>
    <hr>
    <p><strong>Total Distance:</strong> {sum(r['distance'] for r in routes):.1f} km</p>
    <p><strong>Total Cost:</strong> ₨{sum(r['cost'] for r in routes):.2f}</p>
    <p><strong>Vehicles Used:</strong> {len(routes)}</p>
    <p><em>Powered by OSRM real-world routing</em></p>
    </div>
    '''

    m.get_root().html.add_child(folium.Element(legend_html))

    # Save and return file name

    return m


In [58]:
import folium

In [59]:

# Create and save enhanced map
if osrm_routes:
    print("🗺️ Creating OSRM-enhanced route map...")
    osrm_map = create_osrm_map(locations, osrm_routes, demands)
    osrm_map.save('osrm_optimized_routes.html')
    print("✅ OSRM map saved as 'osrm_optimized_routes.html'")

    # Display if possible
    try:
        osrm_map
    except:
        print("💡 Open 'osrm_optimized_routes.html' in your browser!")

🗺️ Creating OSRM-enhanced route map...
✅ OSRM map saved as 'osrm_optimized_routes.html'
